# Loading required libraries

In [1]:
! pip install ipykernel
! pip install -U pip setuptools wheel
! pip install -U spacy[transformers,lookups]==3.0.3
! python -m spacy download en_core_web_trf
! pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
! pip install cupy-cuda113
! pip install scispacy
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_scibert-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.8/668.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 537.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 10.4 MB/s eta 0:00:

# Load Spacy Model

In [2]:
# pip install -U pip setuptools wheel
# pip install -U spacy
! python -m spacy download en_core_web_sm

2023-04-26 19:44:57.452145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 19:45:00.029175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl#egg=en_core_web_sm==3.0.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 80

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import warnings
warnings.filterwarnings('ignore')
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import json
import pandas as pd

In [5]:
# Load the scibert model and create a blank NER model
nlp = spacy.load("en_core_sci_scibert")
ner_model = spacy.blank('en')

# Added NER to the blank NER model
ner = ner_model.add_pipe('ner', last=True)

# Combine the two models
ner_model.vocab.vectors = nlp.vocab.vectors

In [6]:
# Create Examples for training

# Load the training data
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split


train_json_data = pd.read_json('/content/gdrive/MyDrive/data_train.json', orient='records', lines=True)
# split train and test data
train_data, test_data = train_test_split(train_json_data, test_size=0.2, random_state=1)


# Convert training data to Example objects
train_examples = []
labels = []

def remove_overlapping(anns):
    # anns =   [(start, end, label), (start, end, label), ...]

    # sort annotations by start index
    sorted_anns = sorted(anns, key=lambda ann: ann[0])
    # iterate over annotations
    i = 0
    while i < len(sorted_anns) - 1:
        # check if there is an overlap
        if sorted_anns[i][1] > sorted_anns[i + 1][0]:
            # remove the annotation with the shorter duration
            if sorted_anns[i][1] - sorted_anns[i][0] < sorted_anns[i + 1][1] - sorted_anns[i + 1][0]:
                del sorted_anns[i]
            else:
                del sorted_anns[i + 1]
        else:
            i += 1
    return sorted_anns


for row in train_data.iterrows():
    text = row[1]['sentence']
    annotations = row[1]['annotations']

    #anns =  { entities: [(start, end, label), (start, end, label), ...] }
    anns = {"entities": set()}
    for annotation in annotations:
        start = annotation[0]
        end = annotation[1]
        label = annotation[2]
        anns["entities"].add((start, end, label))


    anns["entities"] = list(anns["entities"])
    anns["entities"] = remove_overlapping(anns["entities"])



    for start, end, label in anns['entities']:
        if label not in labels:
            labels.append(label)
    # print(text + '\n' + str(anns) + '\n')
    example = Example.from_dict(nlp.make_doc(text), anns)
    train_examples.append(example)


# # Load the training data
# TRAIN_DATA = [
#     ("Homo sapiens is a species of primates.", {"entities": [(0, 11, "GENUS"), (12, 18, "SPECIES"), (32, 39, "FAMILY"), (40, 47, "ORDER")]}),
#     ("Canis lupus is a species of canine.", {"entities": [(0, 5, "GENUS"), (6, 11, "SPECIES"), (25, 31, "FAMILY")]}),
#     ("The Felidae family includes many species of cats.", {"entities": [(4, 11, "FAMILY")]}),
#     ("The Panthera genus includes lions and tigers.", {"entities": [(4, 11, "GENUS"), (21, 26, "SPECIES"), (31, 37, "SPECIES")]}),
# ]

# # Convert training data to Example objects
# train_examples = []
# labels = []
# for text, anns in TRAIN_DATA:
#     for start, end, label in anns['entities']:
#         if label not in labels:
#             labels.append(label)
#     example = Example.from_dict(nlp.make_doc(text),anns)
#     train_examples.append(example)

# Add the NER labels to the model
for label in labels:
    ner.add_label(label)




## Train the NER model

In [7]:
# Train the NER model
n_iter = 100
batch_size = 8
ner_model.begin_training()
for i in range(n_iter):
    random.shuffle(train_examples)
    batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
    losses = {}
    for batch in batches:
        ner_model.update(batch, losses=losses, drop=0.5)
    print(f"Iteration {i}: Loss={losses['ner']}")

# Save the models to the disk
# ner_model.to_disk('ner_based_on_scibert')
# nlp.to_disk('My_NER_Model')


Iteration 0: Loss=15575.478946454688
Iteration 1: Loss=8120.571012154366
Iteration 2: Loss=7266.712603087622
Iteration 3: Loss=6970.484872309783
Iteration 4: Loss=6682.156946293339
Iteration 5: Loss=6209.875581364895
Iteration 6: Loss=6033.987530399901
Iteration 7: Loss=6009.437913040645
Iteration 8: Loss=5787.204960482878
Iteration 9: Loss=5856.2937716218075
Iteration 10: Loss=5621.080647480637
Iteration 11: Loss=5509.763661774512
Iteration 12: Loss=5442.467791837287
Iteration 13: Loss=5484.020503656186
Iteration 14: Loss=5373.308035461976
Iteration 15: Loss=5414.40814624588
Iteration 16: Loss=5319.395153683787
Iteration 17: Loss=5209.899559610225
Iteration 18: Loss=5272.006535773571
Iteration 19: Loss=5179.483272161615
Iteration 20: Loss=5118.04500554544
Iteration 21: Loss=5090.0567233442835
Iteration 22: Loss=5100.575894477845
Iteration 23: Loss=4958.12964150415
Iteration 24: Loss=5078.945253560903
Iteration 25: Loss=5001.706025382
Iteration 26: Loss=4975.272026248396
Iteration 27: 

FileNotFoundError: ignored

In [8]:
# Save the models to the Google Drive
ner_model.to_disk('/content/gdrive/MyDrive/Models/ner_based_on_scibert')
nlp.to_disk('/content/gdrive/MyDrive/Models/My_NER_Model')

## Testing the model

In [10]:

# Test the NER model

import time

def evaluate(ner_model, data):
    tp_strict = 0
    fp_strict = 0
    fn_strict = 0
    tp_partial = 0
    fp_partial = 0
    fn_partial = 0
    total_time = 0
    for example in data:
        doc = ner_model(example[0])
        gold_ents = example[1]['entities']
        pred_ents = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        gold_set = set(gold_ents)
        pred_set = set(pred_ents)
        tp_strict += len(gold_set & pred_set)
        fp_strict += len(pred_set - gold_set)
        fn_strict += len(gold_set - pred_set)
        for gold in gold_set:
            for pred in pred_set:
                if gold[2] == pred[2]:
                    if pred[0] >= gold[0] and pred[1] <= gold[1]:
                        tp_partial += 1
                        break
                    elif pred[0] < gold[0] and pred[1] > gold[1]:
                        tp_partial += 1
                        break
        fp_partial += len(pred_set - gold_set)
        fn_partial += len(gold_set - pred_set)
        # total_time += doc._.processing_time

    precision_strict = tp_strict / (tp_strict + fp_strict + 1e-10)
    recall_strict = tp_strict / (tp_strict + fn_strict + 1e-10)
    f1_strict = 2 * precision_strict * recall_strict / (precision_strict + recall_strict + 1e-10)

    precision_partial = tp_partial / (tp_partial + fp_partial + 1e-10)
    recall_partial = tp_partial / (tp_partial + fn_partial + 1e-10)
    f1_partial = 2 * precision_partial * recall_partial / (precision_partial + recall_partial + 1e-10)

    processing_time = 0

    return {
        'precision_strict': precision_strict,
        'recall_strict': recall_strict,
        'f1_strict': f1_strict,
        'precision_partial': precision_partial,
        'recall_partial': recall_partial,
        'f1_partial': f1_partial,
        'processing_time': processing_time,
    }

# # Load the saved NER model and test data
# ner_model = spacy.load('new_scibert')


test_examples = []
for row in test_data.iterrows():
    text = row[1]['sentence']
    annotations = row[1]['annotations']
    anns = {"entities": []}
    for annotation in annotations:
        start = annotation[0]
        end = annotation[1]
        label = annotation[2]
        anns["entities"].append((start, end, label))
    example = (text, anns)
    test_examples.append(example)

test_examples = test_examples

# Evaluate the NER model on the test data
start_time = time.time()
metrics = evaluate(ner_model, test_examples)
end_time = time.time()
print(f"Precision (strict): {metrics['precision_strict']:.4f}")
print(f"Recall (strict): {metrics['recall_strict']:.4f}")
print(f"F1-score (strict): {metrics['f1_strict']:.4f}")
print(f"Precision (partial): {metrics['precision_partial']:.4f}")
print(f"Recall (partial): {metrics['recall_partial']:.4f}")
print(f"F1-score (partial): {metrics['f1_partial']:.4f}")
print(f"Total time: {end_time - start_time:.4f}")


Precision (strict): 0.8723
Recall (strict): 0.9276
F1-score (strict): 0.8991
Precision (partial): 0.8745
Recall (partial): 0.9290
F1-score (partial): 0.9009
Total time: 3.9647
